In [ ]:
import os
from pathlib import Path
import sys
import boto3
import logging
import pandas as pd
import json

os.environ["GARPOS_DIR"] = ("./dev/garpos")
# path_cmd = "/Users/franklyndunbar/.PRIDE_PPPAR_BIN"
# os.environ["PATH"] += os.pathsep + path_cmd

from es_sfgtools.pipeline import DataHandler,DATA_TYPE,FILE_TYPE,DataCatalog
from es_sfgtools.garpos_tools import functions as garpos_functions
from es_sfgtools.garpos_tools import schemas as garpos_schemas
from es_sfgtools.utils.gage_data import (
    generate_gage_data_survey_url, 
    list_files_from_gage_data,
    get_survey_file_dict
    )

## Step 1. Establish data storage and catalog directory

In [ ]:
data_dir = Path("./data")
data_handler = DataHandler(working_dir=data_dir)
# Set Logger Location
logging.basicConfig(level=logging.INFO, filemode="w", filename=data_dir/"dev_datahandler.log")

## Step 2. Initial User Input to select Observation Data


#### 2.1 Browse available surveys from the community archive
- Locate the survey of interest in https://gage-data.earthscope.org/archive/seafloor, and note the `network`, `station`, and `survey` names
- In order to use this notebook to process new surveys, the data must first be submitted and made available from the community archive 

#### 2.2 Select a survey and view available files
- User inputs `network`, `station`, and `survey`
- 

In [ ]:
network='aleutian'
site='SEM1'
survey='2018_A_SFG1'

In [ ]:
url = generate_gage_data_survey_url(network, site, survey)
remote_filepaths = list_files_from_gage_data(url)
remote_filepaths
#survey_files = get_survey_file_dict(url)

### 2.3 Add the remote location of discovered files to the data catalog

In [ ]:
dtype_counts = data_handler.add_campaign_data(
    network=network, station=site, survey=survey, remote_filepaths=remote_filepaths)
print(dtype_counts)

## Step 3 Use remote locations to download and store data locally 

In [ ]:
data_handler.download_campaign_data(network=network, station=site, survey=survey)

## Step 4 Process raw data to final product 
![Alt text](garpos_flow.jpg)

### 4.1 Take all acoustic parent files and generate acoustic df's

In [ ]:
data_handler.process_acoustic_data(network=network, station=site, survey=survey)

### 4.2 Take all IMU parent files and generate IMU df's

In [ ]:
data_handler.process_imu_data(network=network, station=site, survey=survey)

### 4.3 Take all GNSS parent files and generate GNSS df's

In [ ]:
data_handler.process_gnss_data(network=network, station=site, survey=survey)

### 4.4 Take all site parent files and generate the site configuration data, which includes ATD offset, sound-velocity DF, and transponder info

In [ ]:
data_handler.process_siteconfig(network=network, station=site, survey=survey)

## Step 5 Merge sessions for GARPOS (each set of temporally identical IMU,Acoustic, and GNSS data frames)

### 5.1 Get a df of all session data

In [ ]:
session_df = data_handler.get_observation_session_data(network=network, station=site, survey=survey,plot=True)
session_df.head()

### 5.2 Generate a dict. of all data corresponding to the temporal groups

In [ ]:

df_paths = data_handler.group_observation_session_data(session_df,timespan='DAY')
print(json.dumps(df_paths, indent=4))

### 5.3 Get Site, ATDOffset, and SVP data

In [ ]:
site_config = data_handler.get_site_config(network=network, station=site, survey=survey)
site_config.name = network + "_" + site + "_" + survey
atd_offset = data_handler.get_atd_offset(network=network, station=site, survey=survey)
svp_data:pd.DataFrame = data_handler.get_svp_data(network=network, station=site, survey=survey)
# Get the first day of data
observables = df_paths["2018-05-14 00:00:00"]


### 5.4 Merge imu, acoustic, and gnss data

In [ ]:
merged_imu = pd.concat([pd.read_csv(path) for path in observables["imu"]])
merged_acoustic = pd.concat([pd.read_csv(path) for path in observables["acoustic"]])
merged_gnss = pd.concat([pd.read_csv(path) for path in observables["gnss"]])


In [ ]:
merged_gnss.head()

In [ ]:
merged_imu.head()


In [ ]:
merged_acoustic.head()

## Step 6 Convert generic data into GARPOS specific formats

In [ ]:

garpos_input = garpos_functions.garpos_input_from_site_obs(
    site_config=site_config,
    atd_offset=atd_offset,
    sound_velocity=svp_data,
    imu_data=merged_imu,
    acoustic_data=merged_acoustic,
    gnss_data=merged_gnss,
)
garpos_fixed = garpos_schemas.GarposFixed()



In [ ]:
for transponder in site_config.transponders:
    print(transponder.position_llh)

## Visualising the data

In [ ]:
data_handler.plot_site_config(site_config=site_config, zoom=7)

In [ ]:
garpos_functions.plot_enu_llh_side_by_side(garpos_input=garpos_input)